<h1 class="font-effect-3d" style="font-family:Ewert; font-size:150%; color:#39d4be;"> Code Modules, Functions, Styling, and Links</h1>

#### [GitHub Repository](https://github.com/OlgaBelitskaya/deep_learning_projects/tree/master/DL_PP2) & [Colaboratory Version](https://colab.research.google.com/drive/1Z9Fz0OOi6bpWvH-H2OhExC9CkGPWBYZz)

In [ ]:
%%html
<style>
@import url('https://fonts.googleapis.com/css?family=Roboto|Ewert&effect=3d');
span {font-family:Roboto; color:black; text-shadow:4px 4px 4px #aaa;}  
a,h4,div.output_area pre{font-family:Roboto; font-size:120%; color:#39d4be;}      
</style>

In [ ]:
import warnings; warnings.filterwarnings('ignore')
import numpy as np,pandas as pd,pylab as pl 
import os,h5py
from skimage.transform import resize
fpath='../input/classification-of-handwritten-letters/'
print(os.listdir("../input"))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from keras.models import Sequential,load_model
from keras.layers import Dense,Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Conv2D,MaxPooling2D,GlobalAveragePooling2D
from keras.applications.vgg19 import VGG19,preprocess_input as vgg19pi
from keras.applications.vgg16 import VGG16,preprocess_input as vgg16pi
from keras.applications.resnet50 import ResNet50,preprocess_input as resnet50pi
from keras.applications.resnet_v2 import ResNet152V2,preprocess_input as preiRN
from keras import __version__; print('keras version:', __version__)
from sklearn import __version__; print('sklearn version:', __version__)

In [ ]:
def ohe(x): 
    out=OneHotEncoder(categories='auto').fit(x.reshape(-1,1))
    return out.transform(x.reshape(-1,1)).toarray().astype('int64')
def tts(X,y): 
    x_train,x_test,y_train,y_test=\
    train_test_split(X,y,test_size=.2,random_state=1)
    n=int(len(x_test)/2)
    x_valid,y_valid=x_test[:n],y_test[:n]
    x_test,y_test=x_test[n:],y_test[n:]
    return x_train,x_valid,x_test,y_train,y_valid,y_test
def history_plot(fit_history):
    keys=list(fit_history.history.keys())[0:4]
    pl.figure(figsize=(12,10)); pl.subplot(211)
    pl.plot(fit_history.history[keys[0]],
            color='slategray',label='train')
    pl.plot(fit_history.history[keys[2]],
            color='#39d4be',label='valid')
    pl.xlabel("Epochs"); pl.ylabel("Loss")
    pl.legend(); pl.grid(); pl.title('Loss Function')     
    pl.subplot(212)
    pl.plot(fit_history.history[keys[1]],
            color='slategray',label='train')
    pl.plot(fit_history.history[keys[3]],
            color='#39d4be',label='valid')
    pl.xlabel("Epochs"); pl.ylabel("Accuracy")    
    pl.legend(); pl.grid(); pl.title('Accuracy'); pl.show()

<h1 class="font-effect-3d" style="font-family:Ewert; font-size:150%; color:#39d4be;"> Preprocessing the Data </h1>

In [ ]:
f=h5py.File(fpath+'LetterColorImages_123.h5','r')
keys=list(f.keys()); keys 

In [ ]:
letters=u'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
images=np.array(f[keys[1]])/255
labels=np.array(f[keys[2]])
pl.figure(figsize=(2,3)); il=10**4
pl.xticks([]); pl.yticks([])
pl.title('Label: %s \n'%letters[labels[il]-1],
         fontsize=18)
pl.imshow(images[il]); pl.show()

In [ ]:
clabels=ohe(labels)
x_train1,x_valid1,x_test1,\
y_train1,y_valid1,y_test1=tts(images,clabels)
sh=[el.shape for el in \
[clabels,x_train1,y_train1,x_valid1,y_valid1,x_test1,y_test1]]
pd.DataFrame(sh).T

<h1 class="font-effect-3d" style="font-family:Ewert; font-size:150%; color:#39d4be;"> Classification Models </h1>
#### VGG16

In [ ]:
fw='../input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
vgg16_base_model=VGG16(weights=fw,include_top=False)
pvx_train1=vgg16_base_model.predict(x_train1)
pvx_valid1=vgg16_base_model.predict(x_valid1)
pvx_test1=vgg16_base_model.predict(x_test1)
pvx_train1=pvx_train1.reshape(-1,1,1,pvx_train1.shape[3])
pvx_valid1=pvx_valid1.reshape(-1,1,1,pvx_valid1.shape[3])
pvx_test1=pvx_test1.reshape(-1,1,1,pvx_test1.shape[3])

In [ ]:
sh=pvx_train1.shape[1:]
def vgg16_model():
    model = Sequential()
    model.add(GlobalAveragePooling2D(input_shape=sh))    
    model.add(Dense(2048))
    model.add(LeakyReLU(alpha=.02))
    model.add(Dropout(.5))        
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=.02))
    model.add(Dropout(.5))    
    model.add(Dense(33,activation='softmax'))    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',metrics=['accuracy'])
    return model
vgg16_model=vgg16_model()

In [ ]:
checkpointer=ModelCheckpoint(filepath='weights.best.hdf5',verbose=2,save_best_only=True)
lr_reduction=ReduceLROnPlateau(monitor='val_loss',patience=10,verbose=2,factor=.5)
estopping=EarlyStopping(monitor='val_loss',patience=20,verbose=2)
vgg16_history=\
vgg16_model.fit(pvx_train1,y_train1,
                validation_data=(pvx_valid1,y_valid1),
                epochs=100,batch_size=128,verbose=2,
                callbacks=[checkpointer,lr_reduction,estopping]);

In [ ]:
history_plot(vgg16_history)
vgg16_model.load_weights('weights.best.hdf5')
vgg16_scores=vgg16_model.evaluate(pvx_test1,y_test1)
print("Accuracy: %.2f%%"%(vgg16_scores[1]*100))
vgg16_scores

#### VGG 19

In [ ]:
fw='../input/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'
vgg19bmodel=VGG19(weights=fw,include_top=False)
x_train1,x_valid1,x_test1,\
y_train1,y_valid1,y_test1=tts(images,clabels)
pvx_train1=vgg19bmodel.predict(x_train1)
pvx_valid1=vgg19bmodel.predict(x_valid1)
pvx_test1=vgg19bmodel.predict(x_test1)
pvx_train1=pvx_train1.reshape(-1,1,1,pvx_train1.shape[3])
pvx_valid1=pvx_valid1.reshape(-1,1,1,pvx_valid1.shape[3])
pvx_test1=pvx_test1.reshape(-1,1,1,pvx_test1.shape[3])

In [ ]:
sh=pvx_train1.shape[1:]
def vgg19_model():
    model = Sequential() 
    model.add(GlobalAveragePooling2D(input_shape=sh))    
    model.add(Dense(2048))
    model.add(LeakyReLU(alpha=.02))
    model.add(Dropout(.5))        
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=.02))
    model.add(Dropout(.5))
    model.add(Dense(33,activation='softmax'))   
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',metrics=['accuracy'])
    return model
vgg19_model=vgg19_model()

In [ ]:
checkpointer=ModelCheckpoint(filepath='weights.best.hdf5',
                             verbose=2,save_best_only=True)
reduction=ReduceLROnPlateau(monitor='val_loss',patience=10,
                            verbose=2,factor=.75)
estopping=EarlyStopping(monitor='val_loss',patience=20,verbose=2)
vgg19_history=\
vgg19_model.fit(pvx_train1,y_train1, 
                validation_data=(pvx_valid1,y_valid1),
                epochs=150,batch_size=128,verbose=2,
                callbacks=[checkpointer,lr_reduction,estopping]);

In [ ]:
history_plot(vgg19_history)
vgg19_model.load_weights('weights.best.hdf5')
vgg19_scores=vgg19_model.evaluate(pvx_test1,y_test1)
print("Accuracy: %.2f%%"%(vgg19_scores[1]*100))
vgg19_scores

#### ResNet152V2

In [ ]:
x_train1,x_valid1,x_test1,\
y_train1,y_valid1,y_test1=tts(images,clabels)
RNbmodel=ResNet152V2(weights='imagenet',include_top=False)
pvx_train1=RNbmodel.predict(x_train1)
pvx_valid1=RNbmodel.predict(x_valid1)
pvx_test1=RNbmodel.predict(x_test1)
pvx_train1=pvx_train1.reshape(-1,1,1,pvx_train1.shape[3])
pvx_valid1=pvx_valid1.reshape(-1,1,1,pvx_valid1.shape[3])
pvx_test1=pvx_test1.reshape(-1,1,1,pvx_test1.shape[3])

In [ ]:
sh=pvx_train1.shape[1:]
def RNmodel():
    model=Sequential()  
    model.add(GlobalAveragePooling2D(input_shape=sh))   
    model.add(Dense(2048))
    model.add(LeakyReLU(alpha=.02))
    model.add(Dropout(.25)) 
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=.02))
    model.add(Dropout(.25))        
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=.02))
    model.add(Dropout(.25))   
    model.add(Dense(33,activation='softmax'))    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',metrics=['accuracy'])
    return model
RNmodel=RNmodel()

In [ ]:
checkpointer=ModelCheckpoint(filepath='weights.best.hdf5',
                             verbose=2,save_best_only=True)
lr_reduction=ReduceLROnPlateau(monitor='val_loss',patience=5,
                               verbose=2,factor=.75)
estopping=EarlyStopping(monitor='val_loss',patience=20,verbose=2)
history=RNmodel.fit(pvx_train1,y_train1, 
                    validation_data=(pvx_valid1,y_valid1), 
                    epochs=100,batch_size=128,verbose=2, 
                    callbacks=[checkpointer,lr_reduction,estopping]);

In [ ]:
history_plot(history)
RNmodel.load_weights('weights.best.hdf5')
RNscores=RNmodel.evaluate(pvx_test1,y_test1)
print("Accuracy: %.2f%%"%(RNscores[1]*100))
RNscores